# Compute Nino3.4 DJF index for each model, and save to file

In [ ]:
# Computes Indexes without caring about branch years

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.signal import detrend
from matplotlib import pyplot as plt
from eofs.xarray import Eof
from scipy import signal
import pandas as pd
import xarray as xr
import xesmf as xe
import intake
import pprint 
import util 

# choose where to load data from:
load_data_from = 'cloud'
#load_data_from = 'glade'

if load_data_from == 'glade':
    col = intake.open_esm_datastore("../catalogs/glade-cmip6.json")
    file = 'available_data.txt'
else:
    col = intake.open_esm_datastore("../catalogs/pangeo-cmip6.json")
    file = 'available_data_cloud.txt'

In [2]:
# pick only models with at least 496 yrs in piControl
minyrs_control = 496;
# models with fewer years often missed future scenarios, so they are not so interesting for us

# load table:
data_table = pd.read_table(file,index_col=0)
models_used = data_table['piControl (yrs)'][data_table['piControl (yrs)'] >= minyrs_control].index
print(models_used)

Index(['BCC-CSM2-MR', 'CanESM5', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'E3SM-1-0',
       'EC-Earth3', 'EC-Earth3-Veg', 'MIROC-ES2L', 'MIROC6', 'HadGEM3-GC31-LL',
       'HadGEM3-GC31-MM', 'UKESM1-0-LL', 'MRI-ESM2-0', 'GISS-E2-1-G', 'CESM2',
       'CESM2-WACCM', 'GFDL-ESM4', 'SAM0-UNICON', 'MCM-UA-1-0'],
      dtype='object')


In [3]:
##### Possible missing data in cloud: #####
# 'E3SM-1-0': historical ensemble members 1 and 4 seems to have some missing years in the end
# 'EC-Earth3': historical r24i1p1f1 has some missing years in the end
# 'MRI-ESM2-0': ensemble members 2-5 for ssp245 has only 16 years (no more years are available at ESGF)
# 'GISS-E2-1-G': 4 of the piControl members contain no data. Same problem on Glade, so probably no data at ESGF either.. 
# 'CESM2-WACCM': 'ssp370': members 2 and 3 end in year 2055 (also at ESGF)
# 'GFDL-ESM4': missing historical in cloud, but this seems to be available on Glade. Glade piControl is missing on of the 100yr files
##### For this model, we would have to load piControl from cloud, then historical from Glade, and ssp's from cloud


## Choose what model to use 

In [103]:
model = models_used[12]
#model = models_used[13]
model

'MRI-ESM2-0'

In [104]:
data_table.loc[model]

piControl (ens.mem.)         1
historical (ens.mem.)        5
ssp126 (ens.mem.)            1
ssp245 (ens.mem.)            5
ssp370 (ens.mem.)            5
ssp585 (ens.mem.)            1
abrupt-4xCO2 (ens.mem.)     13
piControl (yrs)            701
historical (yrs)           165
ssp126 (yrs)                86
ssp245 (yrs)                86
ssp370 (yrs)                86
ssp585 (yrs)                86
abrupt-4xCO2 (yrs)         151
Name: MRI-ESM2-0, dtype: object

In [105]:
# what experiments does this model have that we want to study?
if any(data_table.loc[model][:6] == 'data problem') == False:
    exp_list = [exp[:-11] for exp in data_table.loc[model][:6].index if float(data_table.loc[model][:6][exp]) > 0]
else:
    exp_list = []
    for exp in (data_table.loc[model][:6].index):
        if  (data_table.loc[model][:6][exp] != 'data problem'):
            exp_list = np.append(exp_list, exp[:-11])
print(exp_list)   

['piControl', 'historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']


In [106]:
exp_keys = {}; datasets = {}

for exp in exp_list:
#for exp in [exp_list[1]]:
    print(exp)
    #cat = col.search(experiment_id = exp, source_id = model, variable_id='ts', table_id='Amon', member_id = 'r1i1p1f1')
    cat = col.search(experiment_id = exp, source_id = model, variable_id='ts', table_id='Amon') 
        
    dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
    for key in dset_dict.keys():
        exp_keys[exp] = key
        datasets[key] = dset_dict[key]

exp_keys

piControl
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
historical
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp126
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp245
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp370
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
ssp585
--> The keys in the returned di

{'piControl': 'CMIP.MRI.MRI-ESM2-0.piControl.Amon.gn',
 'historical': 'CMIP.MRI.MRI-ESM2-0.historical.Amon.gn',
 'ssp126': 'ScenarioMIP.MRI.MRI-ESM2-0.ssp126.Amon.gn',
 'ssp245': 'ScenarioMIP.MRI.MRI-ESM2-0.ssp245.Amon.gn',
 'ssp370': 'ScenarioMIP.MRI.MRI-ESM2-0.ssp370.Amon.gn',
 'ssp585': 'ScenarioMIP.MRI.MRI-ESM2-0.ssp585.Amon.gn'}

In [107]:
# load a dataset for manual calendar check:
# if other than noleap, above function must be changed
exp = exp_list[0]; print(exp)
key = exp_keys[exp]
exp_datasets = datasets[key]
members_sorted = exp_datasets.member_id.sortby(exp_datasets.member_id)

ds = exp_datasets.sel(member_id = members_sorted[0])
#print(ds.time)

#if load_data_from == 'glade'
# Time formats for piControl:
# 'BCC-CSM2-MR': cftime.DatetimeNoLeap
# 'FGOALS-g3': cftime.DatetimeNoLeap # missing files
# 'CanESM5': cftime.DatetimeNoLeap # missing ensemble members
# 'CNRM-CM6-1': cftime.DatetimeGregorian
# 'CNRM-ESM2-1': cftime.DatetimeGregorian # ssp126 have too many time points.
# 'E3SM-1-0': cftime.DatetimeNoLeap # missing files
# 'EC-Earth3': Timestamp('2259-01-16 12:00:00'), ... , cftime.DatetimeProlepticGregorian(2759,..
# 'EC-Earth3-Veg': Timestamp('1852-01-16 12:00:00'), ... , cftime.DatetimeProlepticGregorian(2347,..
# 'IPSL-CM6A-LR': Timestamp('1850-01-16 12:00:00'), ... , cftime.DatetimeGregorian(3049,..
# 'MIROC-ES2L': Timestamp('1850-01-16 12:00:00'), ... , cftime.DatetimeGregorian(2349,
# 'MIROC6': cftime.DatetimeGregorian # code below works, but is not accounting for leap years yet
# 'UKESM1-0-LL': cftime.Datetime360Day # code below works, but must be adjusted for 360day calendar
# 'MRI-ESM2-0': cftime.DatetimeProlepticGregorian # More data should be requested
# 'GISS-E2-1-G': cftime.DatetimeNoLeap # ok, but has no ssp (not in ESGF either)
# 'GISS-E2-1-H': cftime.DatetimeNoLeap # ok, but has no ssp (not in ESGF either)
# 'CESM2': cftime.DatetimeNoLeap 
# 'CESM2-WACCM': cftime.DatetimeNoLeap
# 'GFDL-CM4': cftime.DatetimeNoLeap
# 'SAM0-UNICON': cftime.DatetimeNoLeap
if model in ['BCC-CSM2-MR', 'FGOALS-g3', 'CanESM5', 'E3SM-1-0', 'GISS-E2-1-G', 'GISS-E2-1-H', 'CESM2', 'CESM2-WACCM', 'GFDL-CM4', 'SAM0-UNICON', 'GFDL-ESM4', 'MCM-UA-1-0']:
    ds_calendar = 'noleap'
elif model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6']:
    ds_calendar = 'gregorian'
elif model in ['EC-Earth3', 'EC-Earth3-Veg', 'MRI-ESM2-0']:
    ds_calendar = 'proleptic_gregorian'
elif model in ['UKESM1-0-LL', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM']:
    ds_calendar = '360_day'
    
print(ds_calendar, 'calendar')

piControl
proleptic_gregorian calendar


In [108]:
def area_weights(lat_bnds, lon_bnds): 
    # computes exact area weigths assuming earth is a perfect sphere
    lowerlats = np.radians(lat_bnds[:,0]); upperlats = np.radians(lat_bnds[:,1])
    difflon = np.radians(np.diff(lon_bnds[0,:])) # if the differences in longitudes are all the same
    areaweights = difflon*(np.sin(upperlats) - np.sin(lowerlats));
    areaweights /= areaweights.mean()
    return areaweights # list of weights, of same dimension as latitude

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def leap_year(year, calendar='standard'):
    """Determine if year is a leap year"""
    leap = False
    if ((calendar in ['standard', 'gregorian',
        'proleptic_gregorian', 'julian']) and
        (year % 4 == 0)):
        leap = True
        if ((calendar == 'proleptic_gregorian') and
            (year % 100 == 0) and
            (year % 400 != 0)):
            leap = False
        elif ((calendar in ['standard', 'gregorian']) and
                 (year % 100 == 0) and (year % 400 != 0) and
                 (year < 1583)):
            leap = False
    return leap

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def get_dpm(time, calendar='standard'):
    """
    return a array of days per month corresponding to the months provided in `months`
    """
    month_length = np.zeros(len(time), dtype=np.int)

    cal_days = dpm[calendar]

    for i, (month, year) in enumerate(zip(time.month, time.year)):
        month_length[i] = cal_days[month]
        if leap_year(year, calendar=calendar) and month == 2: # the feb-test is missing at the website!
            month_length[i] += 1
    return month_length

In [109]:
# inspiration taken from: http://xarray.pydata.org/en/stable/examples/monthly-means.html

# days per month:
dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
      }

def day_weights(ds, chosen_season = 'DJF', calendar = 'noleap'): # new function
    month_length = xr.DataArray(get_dpm((ds.time.to_index()), calendar=ds_calendar), coords=[ds.time], name='month_length')
    if chosen_season == 'DJF':
        season_months = month_length.where(month_length['time.season'] == season)
        # repeat last December month, and move it to the beginning
        season_months = xr.concat([season_months[-1], season_months], dim = 'time')

        norm_by_annual = season_months[1:].groupby('time.year').mean('time') # make annual mean
        norm_by_monthly = np.concatenate([np.tile(norm_by_annual.values[i], 12) for i in range(len(norm_by_annual.values))])
        # repeat last December month to give it equal length as season_months. Value of last month will not be used.
        norm_by_monthly = np.concatenate([norm_by_monthly, [norm_by_monthly[-1]]])

        weights = season_months/norm_by_monthly
        # make weigths have mean 1 in chosen season for all years
        # can be checked by running weights.rolling(min_periods=3, center=True, time=3).mean()
        # note that these weights start with a December month
    elif chosen_season == 'all':
        
        ##### This code is not tested yet #####
        norm_by_annual = season_months.groupby('time.year').mean('time') # make annual mean
        norm_by_monthly = np.concatenate([np.tile(norm_by.values[i], 12) for i in range(len(norm_by.values))])
        weights = month_length/norm_by_monthly
        # normalized to have mean 1
    # if other season wanted, continue developing this if-test
    
    # NB: normalised weights do not care what numbers are produced for other seasons
    return weights 

In [110]:
latregion = slice(-5,5); lonregion = slice(190, 240) # = 120 W - 170 W
# use larger region before regridding, that adds 5 deg to each border:
larger_latregion = slice(-10,10); larger_lonregion = slice(185, 245)

resolution = 1;
ds_out = xr.Dataset({'lon': (['lon'], np.arange(lonregion.start+resolution/2, lonregion.stop+resolution/2, resolution)),
                     'lat': (['lat'], np.arange(latregion.start+resolution/2, latregion.stop+resolution/2, resolution))
                    }
                   )
    
regr_lat_bnds = np.array([[upper, upper+resolution] for upper in range(latregion.start,latregion.stop)])
regr_lon_bnds = np.array([[upper, upper+resolution] for upper in range(lonregion.start,lonregion.stop)])
area_w = area_weights(regr_lat_bnds, regr_lon_bnds)

season = 'DJF'
lastD = {}

for exp in exp_list:
#for exp in exp_list[:2]:
    key = exp_keys[exp]
    exp_datasets = datasets[key]
    members_sorted = exp_datasets.member_id.sortby(exp_datasets.member_id)
    #for member in [members_sorted.values[0]]: # check for first member only
    for member in members_sorted.values:
        print(exp, member)
        ds = exp_datasets.sel(member_id = member)
        
        # select regional data, perform a regridding, and compute area average
        if model == 'MCM-UA-1-0':
             ds = ds.rename({'longitude': 'lon','latitude': 'lat'}) 
        regional_data = ds.ts.sel(lat = larger_latregion, lon = larger_lonregion)
        regridder = xe.Regridder(regional_data, ds_out, 'bilinear', reuse_weights = True)
        regridded_data = regridder(regional_data)
        area_avg = (regridded_data.transpose('time', 'lon', 'lat') * area_w).mean(dim=['lon', 'lat'])
            
        yrs = int(area_avg.shape[0]/12)
        
        weights = day_weights(area_avg, calendar = ds_calendar)
        # double check that weights are 1 for all seasons
        meanweights = weights.rolling(min_periods=3, center=True, time=3).mean()
        print('years in experiment:', yrs, '    ',  'mean weights all 1?', all(meanweights.dropna(dim = 'time') == 1))
        
        if exp == 'historical':
            # save last december month for each member for use in season mean in first year of ssp exps
            lastD[member] = area_avg[-1] 
            weights = weights[1:] # drop first december month
        elif exp == 'piControl':
            weights = weights[1:] # drop first december month
        elif exp not in ['piControl','historical']: # then it must be future scenario   
            area_avg = xr.concat([lastD[member], area_avg], dim = 'time')  
            weights = weights.assign_coords(time = area_avg.time)
            
        # average over season
        day_weighted_avg = area_avg*weights
        ds_season = day_weighted_avg.where(day_weighted_avg['time.season'] == season) # creates nan in all other months
            
        ds_season3 = ds_season.rolling(min_periods=3, center=True, time=3).mean()
        
        if exp not in ['piControl','historical']:
            # remove nan-value obtained from inserting last december month from historical
            ds_season3 = ds_season3[1:]
        seasonmean = ds_season3.groupby('time.year').mean('time') # make annual mean
        # no information the first year of piControl and historical, since we are missing the december month before
        
        # day-weighted rolling 3-months mean for all months (with seasonal variations)
        #day_weighted_avg_allyear = area_avg*day_weights(yrs, chosen_season = 'all')
        #smoothed_allyear = day_weighted_avg_allyear.rolling(min_periods=3, center=True, time=3).mean()
        
        colname = [(exp, member)]
        
        first_member_piControl = 'r1i1p1f1'
        if model in ['CNRM-CM6-1', 'CNRM-ESM2-1', 'UKESM1-0-LL', 'MIROC-ES2L']:
            first_member_piControl = 'r1i1p1f2'
        elif model in ['GISS-E2-1-G']:
            first_member_piControl = 'r101i1p1f1'
        
        if exp == 'piControl' and member == first_member_piControl:
            # create dataframe for storing all results and make the piControl years the index
            df = pd.DataFrame(seasonmean.values, columns = colname)
        else:
            df_col = pd.DataFrame(seasonmean.values, columns = colname)
            df = pd.merge(df, df_col, left_index=True, right_index=True, how='outer')
        
df.columns = pd.MultiIndex.from_tuples(df.columns, names=['Experiment','Member'])


piControl r1i1p1f1
Reuse existing file: bilinear_18x53_10x50.nc
years in experiment: 701      mean weights all 1? True
historical r1i1p1f1
Reuse existing file: bilinear_18x53_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r2i1p1f1
Reuse existing file: bilinear_18x53_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r3i1p1f1
Reuse existing file: bilinear_18x53_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r4i1p1f1
Reuse existing file: bilinear_18x53_10x50.nc
years in experiment: 165      mean weights all 1? True
historical r5i1p1f1
Reuse existing file: bilinear_18x53_10x50.nc
years in experiment: 165      mean weights all 1? True
ssp126 r1i1p1f1
Reuse existing file: bilinear_18x53_10x50.nc
years in experiment: 86      mean weights all 1? True
ssp245 r1i1p1f1
Reuse existing file: bilinear_18x53_10x50.nc
years in experiment: 86      mean weights all 1? True
ssp245 r2i1p1f1
Reuse existing file: bilinear_18x53

In [111]:
# check values in last December for historical
[lastD[member].values for member in lastD.keys()]

[array(300.4854556),
 array(299.79895231),
 array(300.10204797),
 array(301.15084893),
 array(302.04415665)]

In [112]:
pd.set_option('display.min_rows', 165)
df.iloc[:165]

Experiment   piControl  historical                                      \
Member        r1i1p1f1    r1i1p1f1    r2i1p1f1    r3i1p1f1    r4i1p1f1   
0                  NaN         NaN         NaN         NaN         NaN   
1           299.469245  300.965208  299.909546  301.565703  301.659591   
2           299.488385  300.273215  302.114248  299.420920  300.589829   
3           299.279575  299.853160  298.407704  299.457912  299.201940   
4           302.199837  299.331803  298.184487  300.539837  301.368462   
5           298.848160  301.108471  300.361720  298.731528  299.428859   
6           301.000703  299.886448  301.364145  300.522376  299.191134   
7           300.288711  298.674349  299.525421  301.128944  300.685145   
8           299.912096  301.521803  300.741637  300.129804  301.253270   
9           299.198171  301.281218  300.591893  299.962843  299.960531   
10          300.862938  299.361353  300.336470  302.028044  300.413884   
11          299.437431  300.024263  300.830246  300.429003  300.008893   
12          300.232510  300.485347  301.779093  301.768105  299.743615   
13          299.437612  300.612458  299.712361  299.779291  301.392843   
14          299.895170  299.270708  299.033168  300.888126  300.833649   
15          300.641217  300.003155  300.695901  298.603240  299.758455   
16          302.507617  299.695128  299.828832  301.220854  299.727870   
17          300.109081  299.787978  299.658788  298.694821  301.482658   
18          300.586146  301.909471  298.664796  300.628627  300.306134   
19          300.043643  300.760504  299.978729  300.202371  299.855871   
20          299.668582  299.346045  299.669993  302.230859  299.691080   
21          300.404706  302.414585  301.733755  298.957212  300.995407   
22          299.294934  299.448806  298.583217  298.952344  299.409272   
23          299.686751  297.805381  300.588061  300.115376  299.445306   
24          299.878796  298.724411  300.880146  299.231124  302.096432   
25          300.535992  301.930136  300.333015  301.590210  299.980462   
26          299.781145  300.253443  302.084908  302.076601  299.250758   
27          300.294503  299.646825  301.135330  298.806986  300.429841   
28          299.416284  301.844837  299.100841  299.247644  300.351894   
29          301.777464  299.344721  300.761025  299.227279  301.320116   
..                 ...         ...         ...         ...         ...   
135         300.918687  301.625531  300.680944  301.621408  302.504369   
136         301.046084  302.651371  299.663278  300.411499  299.076580   
137         301.756639  299.663197  302.521796  300.278416  301.229584   
138         301.158492  302.318757  300.124267  302.700275  302.930702   
139         300.587057  299.698253  301.865703  299.809111  299.043445   
140         300.321247  299.556011  302.159330  301.999965  300.369251   
141         299.280518  302.749115  299.659532  299.724065  303.334945   
142         299.537823  299.051124  302.940443  300.937584  298.690931   
143         299.884573  300.301256  301.437726  301.795368  300.443430   
144         300.226770  299.834007  299.158795  299.074213  299.621658   
145         300.090033  302.270097  301.165139  299.425647  302.058747   
146         299.327028  299.359229  299.451656  302.942300  299.491096   
147         299.914416  299.938088  301.923506  299.548592  300.716585   
148         300.658652  302.308221  300.940776  301.780577  302.399146   
149         299.869115  300.127525  302.596356  302.165297  301.471319   
150         299.916108  299.993903  298.072636  300.897293  300.323502   
151         302.567552  303.157397  300.275277  300.586702  301.927457   
152         298.068347  299.886496  301.559410  300.901188  298.803753   
153         300.031910  298.892810  300.150731  302.315521  301.241242   
154         300.392478  301.302631  301.586347  301.006889  302.910765   
155         302.585446  302.405894  300.070835  300.868010  300.780841   
156 

## check that first and last rows of ssp exps look reasonable

In [113]:
#pd.set_option('display.min_rows', 90)
df.iloc[0]

Experiment  Member  
piControl   r1i1p1f1           NaN
historical  r1i1p1f1           NaN
            r2i1p1f1           NaN
            r3i1p1f1           NaN
            r4i1p1f1           NaN
            r5i1p1f1           NaN
ssp126      r1i1p1f1    300.200410
ssp245      r1i1p1f1    300.310199
            r2i1p1f1    299.843927
            r3i1p1f1    300.136171
            r4i1p1f1    301.251879
            r5i1p1f1    302.019614
ssp370      r1i1p1f1    300.274569
            r2i1p1f1    299.885197
            r3i1p1f1    300.159886
            r4i1p1f1    301.267011
            r5i1p1f1    302.042352
ssp585      r1i1p1f1    300.147121
Name: 0, dtype: float64

In [114]:
pd.set_option('display.max_columns', 100)
#df['ssp370'].iloc[85:88]
df.iloc[85:88]

Experiment   piControl  historical                                      \
Member        r1i1p1f1    r1i1p1f1    r2i1p1f1    r3i1p1f1    r4i1p1f1   
85          300.244786  300.256537  302.059803  300.179838  299.979605   
86          302.169094  302.699304  298.817097  300.122214  299.915778   
87          298.738071  300.268154  300.194329  301.500789  301.554677   

Experiment                  ssp126      ssp245                             \
Member        r5i1p1f1    r1i1p1f1    r1i1p1f1 r2i1p1f1 r3i1p1f1 r4i1p1f1   
85          299.701066  303.560034  302.665274      NaN      NaN      NaN   
86          299.516284         NaN         NaN      NaN      NaN      NaN   
87          299.793201         NaN         NaN      NaN      NaN      NaN   

Experiment               ssp370                                      \
Member     r5i1p1f1    r1i1p1f1    r2i1p1f1    r3i1p1f1    r4i1p1f1   
85              NaN  305.376955  303.617692  303.471442  301.023338   
86              NaN         NaN         NaN         NaN         NaN   
87              NaN         NaN         NaN         NaN         NaN   

Experiment                ssp585  
Member       r5i1p1f1   r1i1p1f1  
85          302.52415  305.65126  
86                NaN        NaN  
87                NaN        NaN

## Save data to file

In [115]:
df.to_csv('../Processed_data/Nino3_4_DJF/' + model + '_DJF_nino3_4index.txt')

In [ ]:
# check for missing files

#cat = col.search(source_id = model, experiment_id = 'historical', variable_id='ts', table_id='Amon',member_id = 'r5i1p1f1')
#cat = col.search(source_id = model, experiment_id = 'historical', variable_id='ts', table_id='Amon')
#cat = col.search(source_id = model, experiment_id = 'piControl', variable_id='ts', table_id='Amon')

#cat = col.search(source_id = model, experiment_id = 'ssp126', variable_id='ts', table_id='Amon')
cat = col.search(source_id = model, experiment_id = 'ssp370', variable_id='ts', table_id='Amon')



In [ ]:
cat.df